# BC 1
## Convolutional Neural Network

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join
import cv2
import pandas as pd
import os
import random as rn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
SEED = 321
rn.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
data = "../input/flowers-recognition/flowers/"
folders = os.listdir(data)

In [4]:
image_names = []
train_labels = []
train_images = []

size = 32,32

for folder in folders:
    for file in os.listdir(os.path.join(data,folder)):
        if file.endswith("jpg"):
            image_names.append(os.path.join(data,folder,file))
            train_labels.append(folder)
            img = cv2.imread(os.path.join(data,folder,file))
            im = cv2.resize(img,size)
            train_images.append(im)
        else:
            continue

In [5]:
train = np.array(train_images)

train.shape

(4323, 32, 32, 3)

In [6]:
train = train.astype('float32') / 255.0

In [8]:
label_dummies = pd.get_dummies(train_labels)

labels =  label_dummies.values.argmax(1)

In [9]:
pd.unique(train_labels)

array(['dandelion', 'daisy', 'sunflower', 'tulip', 'rose'], dtype=object)

In [10]:
pd.unique(labels)

array([1, 0, 3, 4, 2])

In [13]:
union_list = list(zip(train, labels))
rn.shuffle(union_list)
train,labels = zip(*union_list)


train = np.array(train)
labels = np.array(labels)

In [14]:
X_train,X_val,y_train,y_val = train_test_split(train,labels, test_size = 0.2)
X_val,X_test,y_val,y_test = train_test_split(X_val,y_val, test_size = 0.5)

In [15]:
#one-hot encoding
y_train_one_hot = to_categorical(y_train)
y_val_one_hot = to_categorical(y_val)
y_test_one_hot = to_categorical(y_test)

In [16]:
#build CNN model
model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dense(5, activation = 'softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(X_train,y_train_one_hot, epochs=5,batch_size=128,validation_data =(X_val,y_val_one_hot))

Epoch 1/5
28/28 [==============================] - 7s 24ms/step - loss: 1.5548 - accuracy: 0.2685 - val_loss: 1.3515 - val_accuracy: 0.3889
Epoch 2/5
28/28 [==============================] - 0s 12ms/step - loss: 1.3710 - accuracy: 0.4122 - val_loss: 1.2289 - val_accuracy: 0.4630
Epoch 3/5
28/28 [==============================] - 0s 12ms/step - loss: 1.1790 - accuracy: 0.4823 - val_loss: 1.1212 - val_accuracy: 0.5278
Epoch 4/5
28/28 [==============================] - 0s 12ms/step - loss: 1.1418 - accuracy: 0.5154 - val_loss: 1.1313 - val_accuracy: 0.5301
Epoch 5/5
28/28 [==============================] - 0s 12ms/step - loss: 1.1503 - accuracy: 0.5023 - val_loss: 1.6471 - val_accuracy: 0.3935


## Model Evaluation

In [18]:
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.4341801385681293